In [ ]:
import pandas as pd
import re

from utils.column_merge_utils import merge_cols_and_place, column_merge, merge_병합종류
from utils.table_merge_utils import parse_legal_addr, parse_street_addr

In [ ]:
# 공동주택현황 원본 엑셀파일이 decorated돼 있어서 컬럼명에 이상이 있음
# dataframe으로 이용하기 위해서 전체 컬럼명을 직접 지정해줘야 함
units_summary_col_names = [
    '순번',
    '자치구',
    '주택유형',
    '입주유형',
    '단지명',
    '도로명주소',
    '법정동주소',
    '동수',
    '합계 세대수',
    '분양 세대수',
    '임대 세대수',
    '관리분류',
    '자치관리, 위탁관리',
    '사용승인일',
    '토지면적(㎡)',
    '관리사무소 전화번호',
    '관리사무소 FAX번호',
    '층별 동수 합계',
    '층별 세대수 합계',
    '동수(5층)',
    '세대수(5층)',
    '동수(6~10층)',
    '세대수(6~10층)',
    '동수(11~15층)',
    '세대수(11~15층)',
    '동수(16~20층)',
    '세대수(16~20층)',
    '동수(21층이상)',
    '세대수(21층이상)',
    '전용면적별 합계',
    '세대수(40㎡이하)',
    '세대수(40㎡~60㎡)',
    '세대수(60㎡~85㎡)',
    '세대수(85㎡~102㎡)',
    '세대수(102㎡~135㎡)',
    '세대수(135㎡초과)',
]

In [ ]:
# 주소 파싱 컬럼명 리스트
legal_addr_cols = ['법정동주소(시도)', '법정동주소(군구)', '법정동주소(동리)', '법정동주소(번지)', '법정동주소(상세)', '법정동주소(중복주소)']
street_addr_cols = ['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)', '도로명주소(상세)', '도로명주소(중복주소)']

# 병합키로 쓰이는 컬럼명 리스트
legal_addr_merge_keys = ['법정동주소(시도)','법정동주소(군구)','법정동주소(동리)','법정동주소(번지)','분양형태']
street_addr_merge_keys = ['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)','분양형태']

In [ ]:
# 중복컬럼이지만 병합과정에서 병합되거나 드랍되는 컬럼은 주석으로 기록함
column_merge_rules =[
    # 단지명
    [['단지분류', '주택유형'], ['단지분류']],
    # 법정동주소
    # 도로명주소
    # 사용승인일
    # [['분양형태', '입주유형'], ['분양형태']], 매칭키로 사용
    [['세대수', '합계 세대수'], ['세대수']],
    [['분양세대수', '분양 세대수'], ['분양세대수']],
    [['임대세대수', '임대 세대수'], ['임대세대수']],
    [['관리방식', '관리분류'], ['관리방식']],
]

# 파일 읽기

In [ ]:
master_excel_path = '../input/apt_blds_병합.xlsx'   # 건축물대장, kapt, 서울시공공주택아파트정보 병합본
master_csv_path = '../input/apt_blds_병합.csv'

units_summary_excel_path = '../input/서울시 공동주택 현황(2024.12.31).xlsx' # 세대수 정리본
units_summary_csv_path = '../input/units_summary.csv'

In [ ]:
# 엑셀파일을 csv 파일로 변환
# units_summary = pd.read_excel(
#     units_summary_excel_path, 
#     sheet_name=2,
#     header=4)
# units_summary = units_summary.dropna(axis=1, how='all')
# units_summary.columns = units_summary_col_names
# units_summary.to_csv(units_summary_csv_path, index=False)

In [ ]:
master = pd.read_csv(master_csv_path)
print(f"master : {len(master)}")

units_summary = pd.read_csv(units_summary_csv_path)
print(f"units_summary : {len(units_summary)}")

# 전처리

In [ ]:
# 원본 테이블에 완전 동일한 컬럼이 있으므로 삭제
units_summary = units_summary.drop(columns=['층별 동수 합계', '층별 세대수 합계', '전용면적별 합계'])

# 매칭키로 사용하기 위해 컬럼명 변경
units_summary = units_summary.rename(columns={'입주유형':'분양형태'})

In [ ]:
# id 생성
master.insert(0, 'master_id', range(1,len(master)+1))
units_summary.insert(0, 'units_summary_id', range(1,len(units_summary)+1))

# ['병합종류'] 컬럼 수정
master=master.rename(columns={'병합종류':'master_병합종류'})
units_summary.insert(0, 'units_summary_병합종류', '4번')

# 주소 파싱

In [ ]:
# master의 주소는 이미 파싱돼 있음. units_summary만 파싱
units_summary[legal_addr_cols] = units_summary['법정동주소'].apply(
    lambda x: pd.Series(parse_legal_addr(x))
)

units_summary[street_addr_cols] = units_summary['도로명주소'].apply(
    lambda x: pd.Series(parse_street_addr(x))
)

In [ ]:
# 매칭키는 전부 string으로
for key in legal_addr_merge_keys + street_addr_merge_keys:
    if key in master.columns: master[key] = master[key].astype('string')
    if key in units_summary.columns: units_summary[key] = units_summary[key].astype('string')

# 병합

In [ ]:
def merge_two_df(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    on_list: list[str]
    ):
    suffix = 'suffix'
    
    merged = df1.merge( # OUTER JOIN
        df2,
        on = on_list,
        how='outer',
        suffixes=('',suffix),   # 컬럼명이 같은 경우 suffix 붙임
        indicator=True          # ['_merge'] 컬럼 추가
    )
    
    cols_to_column_merge = {c.removesuffix(suffix):c for c in merged.columns if c.endswith(suffix)}
    for col1, col2 in cols_to_column_merge.items():
        merged = merge_cols_and_place(merged, [col1,col2],[col1])
    
    matched = ( # 양쪽 매칭에 성공한 것
        merged.loc[merged['_merge']=='both']
        .drop(columns=['_merge'])
        .copy()
    )
    left_merged = ( # LEFT JOIN
        merged.loc[merged['_merge']=='left']
        .drop(columns=['_merge'])
        .copy()
    )
    right_merged = (# RIGHT JOIN
        merged.loc[merged['_merge']=='right']
        .drop(columns=['_merge'])
        .copy()
    )
    
    unmatched_df1 = (   # 매칭에 실패한 데이터
        merged.loc[merged['_merge']=='left_only']
        .copy()
    )
    unmatched_df1 = unmatched_df1.drop( # 원본 테이블의 형식으로 되돌리기
        columns=[c for c in unmatched_df1.columns if c not in df1.columns]
    )
    unmatched_df2 = (
        merged.loc[merged['_merge']=='right_only']
        .copy()
    )
    unmatched_df2 = unmatched_df2.drop(
        columns=[c for c in unmatched_df2.columns if c not in df2.columns]
    )
    
    return merged, matched, left_merged, right_merged, unmatched_df1, unmatched_df2

In [ ]:
# 매칭에 포함시킬 master 데이터
master_target = master[
    master['master_병합종류'].str.contains('2번|3번')   # 2번 혹은 3번 데이터를 가지는 row만 매칭에 포함
]
print(f"master_target : {len(master_target)}")

# 법정동주소로 매칭
merged_on_legal_addr, matched_on_legal_addr, left_merged, right_merged, unmatched_master, unmatched_units_summary = merge_two_df(
    master_target, units_summary, legal_addr_merge_keys
)
print(f"matched_on_legal_addr : {len(matched_on_legal_addr)}")

# 도로명주소로 매칭
merged_on_street_addr, matched_on_street_addr, left_merged, right_merged, unmatched_master, unmatched_units_summary = merge_two_df(
    master_target, unmatched_units_summary, street_addr_merge_keys
)
print(f"matched_on_street_addr : {len(matched_on_street_addr)}")
print(f"unmatched_units_summary : {len(unmatched_units_summary)}")

matched = pd.concat([
    matched_on_legal_addr,matched_on_street_addr
    ],ignore_index=True)
matched = matched.drop_duplicates().reset_index(drop=True)  # 중복 제거
print(f"matched : {len(matched)}")

unmatched_master = master[  # 전체 master 중 매칭되지 않은 것
    ~master['master_id'].isin(matched['master_id'])]
merged = pd.concat([
    matched,unmatched_master,unmatched_units_summary
    ],ignore_index=True)
merged = merged.drop_duplicates().reset_index(drop=True)

In [ ]:
legal_addr_merge_keys.remove('분양형태')
street_addr_merge_keys.remove('분양형태')

# 총괄표제부만 매칭에 참여
master_target = master[master['그룹 종류']==0]  
print(f"master_target : {len(master_target)}")

# 법정동주소로 매칭
merged_on_legal_addr, matched_on_legal_addr, left_merged, right_merged, unmatched_master, unmatched_units_summary = merge_two_df(
    master_target, unmatched_units_summary, legal_addr_merge_keys
)
print(f"matched_on_legal_addr : {len(matched_on_legal_addr)}")

# 도로명주소로 매칭
merged_on_street_addr, matched_on_street_addr, left_merged, right_merged, unmatched_master, unmatched_units_summary = merge_two_df(
    master_target, unmatched_units_summary, street_addr_merge_keys
)
print(f"matched_on_street_addr : {len(matched_on_street_addr)}")
print(f"unmatched_units_summary : {len(unmatched_units_summary)}")


matched = pd.concat([
    matched_on_legal_addr,matched_on_street_addr,matched    # 앞의 결과와 concat
    ],ignore_index=True)
matched = matched.drop_duplicates().reset_index(drop=True)  # 중복 제거

unmatched_master = master[  # 전체 master 중 매칭되지 않은 것
    ~master['master_id'].isin(matched['master_id'])]
merged = pd.concat([
    matched, unmatched_master, unmatched_units_summary
])

print(f"matched : {len(matched)}")

In [ ]:
# 총괄표제부를 가지지 않는 일반표제부만 매칭에 참여
master_target = master[master['그룹 종류']==2]  
print(f"master_target : {len(master_target)}")

# 법정동주소로 매칭
merged_on_legal_addr, matched_on_legal_addr, left_merged, right_merged, unmatched_master, unmatched_units_summary = merge_two_df(
    master_target, unmatched_units_summary, legal_addr_merge_keys
)
print(f"matched_on_legal_addr : {len(matched_on_legal_addr)}")

# 도로명주소로 매칭
merged_on_street_addr, matched_on_street_addr, left_merged, right_merged, unmatched_master, unmatched_units_summary = merge_two_df(
    master_target, unmatched_units_summary, street_addr_merge_keys
)
print(f"matched_on_street_addr : {len(matched_on_street_addr)}")
print(f"unmatched_units_summary : {len(unmatched_units_summary)}")


matched = pd.concat([
    matched_on_legal_addr,matched_on_street_addr,matched    # 앞의 결과와 concat
    ],ignore_index=True)
matched = matched.drop_duplicates().reset_index(drop=True)  # 중복 제거

unmatched_master = master[  # 전체 master 중 매칭되지 않은 것
    ~master['master_id'].isin(matched['master_id'])]
merged = pd.concat([
    matched, unmatched_master, unmatched_units_summary
])

print(f"matched : {len(matched)}")
print(f"merged : {len(merged)}")

# 컬럼 병합

In [ ]:
merged = column_merge(merged, column_merge_rules)

#### 병합종류 컬럼 병합

In [ ]:
merged = merge_병합종류(merged, 'master_병합종류', 'units_summary_병합종류')

# 테이블 정리

In [ ]:
# 불필요한 컬럼 제거
unnamed_columns = [c for c in merged.columns if "Unnamed" in c]
merged = merged.drop(columns=['master_병합종류', 'units_summary_병합종류']+unnamed_columns)

In [ ]:
# 컬럼 위치 조정
front_cols = ['병합종류', 'master_id', 'units_summary_id', 'apt_id', 'kapt_id', 'sapt_id', 'blds_id'] + legal_addr_cols + street_addr_cols

other_cols = [c for c in merged.columns if c not in front_cols]

merged = merged[front_cols + other_cols]

# 결과

In [ ]:
merged.to_csv("../output/apt_blds_unitssummary_병합.csv", index=False)
print("csv created")
merged.to_excel("../output/apt_blds_unitssummary_병합.xlsx", index=False)
print("excel created")
unmatched_units_summary.to_excel("../interim/unmatched_units_summary.xlsx")